In [2]:
import os
import numpy as np
import laspy
import pdal
import json
import scipy
import copy
from tqdm import tqdm
from time import time
import itertools
from pyproj import CRS

In [5]:
las = laspy.read(r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2025\functional\Barmasse_2025_AllScans_Raw_Sub2cm_flatten_stripes_10_m\Barmasse_2025_AllScans_Raw_Sub2cm_flatten_stripe_45.laz")
print(len(las))
print(len(set(las.id_point)))

4063806
4063806


In [16]:
las = laspy.read(r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2023\test2\Barmasse_2025_AllScans_Raw_Sub2cm_with_id_tiles_overlap\Barmasse_2025_AllScans_Raw_Sub2cm_with_id_tile_0_0.laz")
print(las.x.min())

-79501.86


In [8]:
las = laspy.read(r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2023\test2\Barmasse_2025_AllScans_Raw_Sub2cm_aligned.laz")


In [ ]:
# header = laspy.LasHeader(point_format=las.header.point_format, version=las.header.version)
las.header.add_crs(CRS.from_epsg(2056))
# las.header.assign_crs(CRS.from_epsg(2056))
las.write(r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2023\test2\Barmasse_2025_AllScans_Raw_Sub2cm_aligned_with_crs.laz")


AttributeError: No attribute assign_crs in LasHeader

In [10]:
epsg = las.header.parse_crs()
print(epsg)

None


In [5]:
src_original = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2023\test\Barmasse_2023_AllScans_Raw_Georef.laz"
tile_size = 500
overlap = 50
grid_size = 10
stripe_width = 10
ext = 'laz'

In [6]:
src_with_id = src_original.split('.laz')[0] + "_with_id.laz"
src_folder_tiles_wo_overlap = os.path.join(os.path.dirname(src_with_id), os.path.basename(src_with_id).split('.laz')[0] + "_tiles_no_overlap")
src_folder_tiles_w_overlap = os.path.join(os.path.dirname(src_with_id), os.path.basename(src_with_id).split('.laz')[0] + "_tiles_overlap")
src_folder_flatten_tiles_w_overlap = os.path.join(os.path.dirname(src_with_id), os.path.basename(src_with_id).split('.laz')[0] + "_flatten")
src_flatten_file = os.path.join(src_original.split('.laz')[0] + "_flatten.laz")
src_folder_stripes = os.path.join(os.path.dirname(src_flatten_file), os.path.basename(src_flatten_file).split('.laz')[0] + "_stripes")


In [7]:
# loading
start_preprocess_time = time()
laz_original = laspy.read(src_original)
print(len(laz_original))

113730157


In [8]:
# utils
def tilling(src_input, src_target, tile_size, overlap=0, verbose=True):
    """
    Tile the input LiDAR file into square tiles using PDAL and store them in the destination folder.

    Args:
        - verbose (bool): Whether to print verbose status updates.

    Returns:
        - None: Splits the input file into tiles and saves them in the destination folder.
    """

    print(f"Start tilling (with overlap = {overlap}m)...")
    os.makedirs(src_target, exist_ok=True)
    
    # compute the estimate number of tiles
    if verbose:
        print("Computing the estimated number of tiles...")
    original_file = laspy.read(src_input)
    x_min = original_file.x.min()
    x_max = original_file.x.max()
    y_min = original_file.y.min()
    y_max = original_file.y.max() 
    if verbose:
        print('Done!')


    
    # output_pattern = "tile_{i}_{j}.laz"
    output_pattern = os.path.join(
        src_target, 
        os.path.basename(src_input).split('.')[0] + "_tile_{i}_{j}.laz",
        )

    x_steps = int((x_max - x_min) / tile_size) + 1
    y_steps = int((y_max - y_min) / tile_size) + 1
    combinations = list(itertools.product(range(x_steps), range(y_steps)))
    for _, (i,j) in tqdm(enumerate(combinations), total=len(combinations)):
        x0 = x_min + i * tile_size - overlap
        x1 = x_min + (i + 1) * tile_size + overlap
        y0 = y_min + j * tile_size - overlap
        y1 = y_min + (j + 1) * tile_size + overlap

        bounds = f"([{x0},{x1}],[{y0},{y1}])"
        pipeline_json = {
            "pipeline": [
                # src_input,
                {
                    "type": "readers.las",
                    "filename": src_input,
                    "extra_dims": "id_point=uint32"
                },
                {"type": "filters.crop", "bounds": bounds},
                {
                    "type": "writers.las",
                    "filename": output_pattern.format(i=i, j=j),
                    "extra_dims": "id_point=uint32"
                }
            ]
        }

        pipeline = pdal.Pipeline(json.dumps(pipeline_json))
        pipeline.execute()


def stripes_file(src_input_file, src_output, dims, do_keep_existing=False):
    [tile_size_x, tile_size_y] = dims
    laz = laspy.read(src_input_file)
    # output_folder = os.path.join(src_output, f"{os.path.basename(src_input_file).split('.laz')[0]}_stripes_{tile_size_x}_{tile_size_y}")
    os.makedirs(src_output, exist_ok=True)

    xmin, xmax, ymin, ymax = np.min(laz.x), np.max(laz.x), np.min(laz.y), np.max(laz.y)

    x_edges = np.arange(xmin, xmax, tile_size_x)
    y_edges = np.arange(ymin, ymax, tile_size_y)


    for i, x0 in enumerate(x_edges):
        print(f"Column {i+1} / {len(x_edges)}:")
        for j, y0 in tqdm(enumerate(y_edges), total=len(y_edges)):
            output = os.path.join(src_output,f"{os.path.basename(src_input_file).split('.laz')[0]}_stripe_{i}_{j}.laz")
            if do_keep_existing and os.path.exists(output):
                continue
            x1 = x0 + tile_size_x
            y1 = y0 + tile_size_y
            bounds = f"([{x0},{x1}],[{y0},{y1}])"
            pipeline_json = {
                "pipeline": [
                    src_input_file,
                    {"type": "filters.crop", "bounds": bounds},
                    {"type": "writers.las", "filename": output, 'extra_dims': 'all'}
                ]
            }

            pipeline = pdal.Pipeline(json.dumps(pipeline_json))
            pipeline.execute()


def stripes_file_fast(src_input_file, src_output, dims, do_keep_existing=False):
    [tile_size_x, tile_size_y] = dims
    laz = laspy.read(src_input_file)
    # output_folder = os.path.join(src_output, f"{os.path.basename(src_input_file).split('.laz')[0]}_stripes_{tile_size_x}_{tile_size_y}")
    os.makedirs(src_output, exist_ok=True)

    xmin, xmax, ymin, ymax = np.min(laz.x), np.max(laz.x), np.min(laz.y), np.max(laz.y)

    x_edges = np.arange(xmin, xmax, tile_size_x)
    y_edges = np.arange(ymin, ymax, tile_size_y)

    list_bounds = []
    for i, x0 in enumerate(x_edges):
        print(f"Column {i+1} / {len(x_edges)}:")
        for j, y0 in tqdm(enumerate(y_edges), total=len(y_edges)):
            output = os.path.join(src_output,f"{os.path.basename(src_input_file).split('.laz')[0]}_stripe_{i}_{j}.laz")
            if do_keep_existing and os.path.exists(output):
                continue
            x1 = x0 + tile_size_x
            y1 = y0 + tile_size_y
            bounds = f"([{x0},{x1}],[{y0},{y1}])"
            # str_list_bounds += f"[{bound[0]},{bound[1]}],"
            
            list_bounds.append(bounds)

    # str_list_bounds = "("
    # for bound in list_bounds:
    # str_list_bounds = str_list_bounds[:-1] + ")"
    # print(str_list_bounds)
    print(len(list_bounds))
    output_pattern = os.path.join(
            src_output, 
            os.path.basename(src_input_file).split('.')[0] + "_stripe_#.laz",
            )
    pipeline_json = {
        "pipeline": [
            {
                "type": "readers.las",
                "filename": src_input_file,
                "extra_dims": "id_point=uint32"
            },
            {
                "type": "filters.crop", 
                "bounds": list_bounds
            },
            {
                "type": "writers.las", 
                "filename": output_pattern, 
                'extra_dims': "id_point=uint32"
            }
        ]
    }

    pipeline = pdal.Pipeline(json.dumps(pipeline_json))
    pipeline.execute()


def remove_duplicates(laz_file, decimals=2):
    """
    Removes duplicate points from a LAS/LAZ file based on rounded 3D coordinates.

    Args:
        - laz_file (laspy.LasData): Input LAS/LAZ file as a laspy object.
        - decimals (int, optional): Number of decimals to round the coordinates for duplicate detection. Defaults to 2.

    Returns:
        - laspy.LasData: A new laspy object with duplicate points removed.
    """
        
    coords = np.round(np.vstack((laz_file.x, laz_file.y, laz_file.z)).T, decimals)
    _, unique_indices = np.unique(coords, axis=0, return_index=True)
    mask = np.zeros(len(coords), dtype=bool)
    mask[unique_indices] = True

    # Create new LAS object
    header = laspy.LasHeader(point_format=laz_file.header.point_format, version=laz_file.header.version)
    new_las = laspy.LasData(header)

    for dim in laz_file.point_format.dimension_names:
        setattr(new_las, dim, getattr(laz_file, dim)[mask])

    return new_las

def match_pointclouds(laz1, laz2):
    """Sort laz2 to match the order of laz1 without changing laz1's order.

    Args:
        laz1: laspy.LasData object (reference order)
        laz2: laspy.LasData object (to be sorted)
    
    Returns:
        laz2 sorted to match laz1
    """
    # Retrieve and round coordinates for robust matching
    coords_1 = np.round(np.vstack((laz1.x, laz1.y, laz1.z)), 2).T
    coords_2 = np.round(np.vstack((laz2.x, laz2.y, laz2.z)), 2).T

    # Verify laz2 is of the same size as laz1
    assert len(coords_2) == len(coords_1), "laz2 should be a subset of laz1"

    # Create a dictionary mapping from coordinates to indices
    coord_to_idx = {tuple(coord): idx for idx, coord in enumerate(coords_1)}

    # Find indices in laz1 that correspond to laz2
    matching_indices = []
    failed = 0
    for coord in coords_2:
        try:
            matching_indices.append(coord_to_idx[tuple(coord)])
        except Exception as e:
            failed += 1

    matching_indices = np.array([coord_to_idx[tuple(coord)] for coord in coords_2])

    # Sort laz2 to match laz1
    sorted_indices = np.argsort(matching_indices)

    # Apply sorting to all attributes of laz2
    laz2.points = laz2.points[sorted_indices]

    return laz2  # Now sorted to match laz1


def flattening_tile(tile_src, tile_new_original_src, grid_size=10, do_save_floor=False, do_keep_existing=False, verbose=True):
    """
    Flattens a tile by interpolating the ground surface and subtracting it from the original elevation.

    Args:
        - tile_src (str): Path to the input tile in LAS/LAZ format.
        - tile_new_original_src (str): Path to save the resized original tile after filtering.
        - grid_size (int, optional): Size of the grid in meters for local interpolation. Defaults to 10.
        - verbose (bool, optional): Whether to display progress and debug information. Defaults to True.

    Returns:
        - None: Saves the floor and flattened versions of the tile and updates the original file.
    """
    if os.path.exists(tile_new_original_src) and do_keep_existing:
        if verbose:
            print(f"Skipping. {tile_new_original_src} exists already")
        return
    # Load file
    laz = laspy.read(tile_src)
    init_len = len(laz)
    if init_len == 0:
        return
    # laz = remove_duplicates(laz)
    if verbose:
        print(f"Removing duplicates: From {init_len} to {len(laz)}")
    # laz.write(tile_new_original_src)
    
    points = np.vstack((laz.x, laz.y, laz.z)).T
    points_flatten = copy.deepcopy(points)
    points_interpolated = copy.deepcopy(points)

    # Divide into tiles and find local minimums
    x_min, y_min = np.min(points[:, :2], axis=0)
    x_max, y_max = np.max(points[:, :2], axis=0)

    x_bins = np.append(np.arange(x_min, x_max, grid_size), x_max)
    y_bins = np.append(np.arange(y_min, y_max, grid_size), y_max)

    grid = {i:{j:[] for j in range(y_bins.size - 1)} for i in range(x_bins.size -1)}
    for _, (px, py, pz) in tqdm(enumerate(points), total=len(points), desc="Creating grid", disable=verbose==False):
        xbin = np.clip(0, (px - x_min) // grid_size, x_bins.size - 2)
        ybin = np.clip(0, (py - y_min) // grid_size, y_bins.size - 2)
        try:
            grid[xbin][ybin].append((px, py, pz))
        except Exception as e:
            print(xbin)
            print(ybin)
            print(x_bins)
            print(y_bins)
            print(grid.keys())
            print(grid[0].keys())
            raise e


    # Create grid_min
    grid_used = np.zeros((x_bins.size - 1, y_bins.size - 1))
    lst_grid_min = []
    lst_grid_min_pos = []
    for x in grid.keys():
        for y in grid[x].keys():
            if np.array(grid[x][y]).shape[0] > 0:
                grid_used[x, y] = 1
                lst_grid_min.append(np.min(np.array(grid[x][y])[:,2]))
                arg_min = np.argmin(np.array(grid[x][y])[:,2])
                lst_grid_min_pos.append(np.array(grid[x][y])[arg_min,0:2])

                # test if border
                if x == list(grid.keys())[0]:
                    lst_grid_min.append(np.min(np.array(grid[x][y])[:,2]))
                    lst_grid_min_pos.append(np.array(grid[x][y])[arg_min,0:2] - [5, 0])
                if x == list(grid.keys())[-1]:
                    lst_grid_min.append(np.min(np.array(grid[x][y])[:,2]))
                    lst_grid_min_pos.append(np.array(grid[x][y])[arg_min,0:2] + [5, 0])
                if y == list(grid[x].keys())[0]:
                    lst_grid_min.append(np.min(np.array(grid[x][y])[:,2]))
                    lst_grid_min_pos.append(np.array(grid[x][y])[arg_min,0:2] - [0, 5])
                if y == list(grid[x].keys())[-1]:
                    lst_grid_min.append(np.min(np.array(grid[x][y])[:,2]))
                    lst_grid_min_pos.append(np.array(grid[x][y])[arg_min,0:2] + [0, 5])
            else:
                grid_used[x, y] = 0
    arr_grid_min_pos = np.vstack(lst_grid_min_pos)
    if verbose:
        print("Resulting grid:")
        print(arr_grid_min_pos.shape)
        print(grid_used)

    # Interpolate
    points_xy = np.array(points)[:,0:2]
    interpolated_min_z = scipy.interpolate.griddata(arr_grid_min_pos, np.array(lst_grid_min), points_xy, method="cubic", fill_value=-1)

    # # Fill NaNs with nearest neighbor interpolation
    # nan_mask = np.isnan(interpolated_min_z)
    # x = np.array(points)[:,0]
    # y = np.array(points)[:,1]
    # z = np.array(points)[:,2]

    # if np.any(nan_mask):
    #     interpolated_min_z[nan_mask] = scipy.interpolate.griddata(arr_grid_min_pos, np.array(lst_grid_min), (x[nan_mask], y[nan_mask]), method='nearest')

    mask_valid = np.array([x != -1 for x in list(interpolated_min_z)])
    points_interpolated = points_interpolated[mask_valid]
    points_interpolated[:, 2] = interpolated_min_z[mask_valid]

    if verbose:
        print("Interpolation:")
        print(f"Original number of points: {points.shape[0]}")
        print(f"Interpollated number of points: {points_interpolated.shape[0]} ({int(points_interpolated.shape[0] / points.shape[0]*100)}%)")

    # save floor
    filtered_points = {dim: getattr(laz, dim)[mask_valid] for dim in laz.point_format.dimension_names}
    header = laspy.LasHeader(point_format=laz.header.point_format, version=laz.header.version)
    new_las = laspy.LasData(header)

    #   _Assign filtered and modified data
    for dim, values in filtered_points.items():
        setattr(new_las, dim, values)
    setattr(new_las, 'x', points_interpolated[:,0])
    setattr(new_las, 'y', points_interpolated[:,1])
    setattr(new_las, 'z', points_interpolated[:,2])

    #   _Save new file
    # floor_dir = os.path.join(os.path.dirname(tile_src), 'floor')
    # os.makedirs(floor_dir, exist_ok=True)
    # new_las.write(os.path.join(floor_dir, os.path.basename(tile_src).split('.laz')[0] + f"_floor_{grid_size}m.laz"))
    if do_save_floor:
        new_las.write(tile_new_original_src.split('.laz')[0] + "_floor.laz")
        if verbose:
            print("Saved file: ", tile_new_original_src.split('.laz')[0] + "_floor.laz")
            # print("Saved file: ", os.path.join(floor_dir, os.path.basename(tile_src).split('.laz')[0] + f"_floor_{grid_size}m.laz"))

    # Flatten
    points_flatten = points_flatten[mask_valid]
    points_flatten[:,2] = points_flatten[:,2] - points_interpolated[:,2]

    filtered_points = {dim: getattr(laz, dim)[mask_valid] for dim in laz.point_format.dimension_names}
    header = laspy.LasHeader(point_format=laz.header.point_format, version=laz.header.version)
    header.point_count = 0
    new_las = laspy.LasData(header)


    #   _Assign filtered and modified data
    for dim, values in filtered_points.items():
        setattr(new_las, dim, values)

    setattr(new_las, 'x', points_flatten[:,0])
    setattr(new_las, 'y', points_flatten[:,1])
    setattr(new_las, 'z', points_flatten[:,2])

    #   _Save new file
    # flatten_dir = os.path.join(os.path.dirname(tile_src), 'flatten')
    # os.makedirs(flatten_dir, exist_ok=True)
    # new_las.write(os.path.join(os.path.dirname(tile_src), os.path.basename(tile_src).split('.laz')[0] + f"_flatten_{grid_size}m.laz"))
    new_las.write(tile_new_original_src.split('.laz')[0] + "_flatten.laz")
    if verbose:
        print("Saved file: ", tile_new_original_src.split('.laz')[0] + "_flatten.laz")

    # Resize original file
    laz.points = laz.points[mask_valid]
    laz.write(tile_new_original_src)
    if verbose:
        print("Saved file: ", tile_new_original_src)


def flattening(src_tiles, src_new_tiles, grid_size=10, verbose=True, do_keep_existing=False, verbose_full=False):
    """
    Applies the flattening process to all tiles in a directory using grid-based ground surface estimation.

    Args:
        - src_tiles (str): Path to the directory containing original tiles.
        - src_new_tiles (str): Path to the directory where resized tiles will be saved.
        - grid_size (int, optional): Size of the grid in meters for interpolation. Defaults to 10.
        - verbose (bool, optional): Whether to show a general progress bar. Defaults to True.
        - verbose_full (bool, optional): Whether to print detailed info per tile. Defaults to False.

    Returns:
        - None: Processes and saves flattened tiles into their respective folders.
    """
    
    print("Starting flattening:")
    list_tiles = [x for x in os.listdir(src_tiles) if x.endswith('.laz')]
    for _, tile in tqdm(enumerate(list_tiles), total=len(list_tiles), desc="Processing", disable=verbose==False):
        if verbose_full:
            print("Flattening tile: ", tile)
        
        flattening_tile(
            tile_src=os.path.join(src_tiles, tile), 
            tile_new_original_src=os.path.join(src_new_tiles, tile),
            grid_size=grid_size,
            do_keep_existing=do_keep_existing,
            verbose=verbose_full,
            )

# # test striping fast
# src_file = r"D:\GitHubProjects\PointCloudUtils\data\test_big_tiling\flattening_10m\test\SwissSurface_2019_2538_1154_tile_67_with_id.laz"
# src_folder_stripes = os.path.join(os.path.dirname(src_file), os.path.basename(src_file).split('.laz')[0] + "_stripes")
# laz_original = laspy.read(src_file)
# x_span = laz_original.x.max() - laz_original.x.min()
# y_span = laz_original.y.max() - laz_original.y.min()
# dims = [x_span, y_span]

# dims[np.argmin([x_span, y_span])] = 10

# print("Creation of stripes:")
# stripes_file_fast(src_file, src_folder_stripes, dims, do_keep_existing=True)

# print(len(laz_original))
# print(len(set(laz_original.id_point)))
# print(max(set(laz_original.id_point)))

In [ ]:
src_file = r"D:\GitHubProjects\PointCloudUtils\data\test_big_tiling\flattening_10m\Barmasse_2023_AllScans_Raw_Georef_flatten_stripes\Barmasse_2023_AllScans_Raw_Georef_flatten_stripe_4.laz"
laz_original = laspy.read(src_file)
print(len(laz_original))
print(len(set(laz_original.id_point)))
print(max(set(laz_original.id_point)))

52238
52238
98134879


In [16]:
# adding an id to the points
id_point = np.arange(len(laz_original))
laz_original.add_extra_dim(laspy.ExtraBytesParams('id_point', type="uint32"))
laz_original.id_point = id_point

src_with_id = src_original.split(f'.{ext}')[0] + f"_with_id.{ext}"
laz_original.write(src_with_id)

In [8]:
# tiles with overlap
src_folder_tiles_w_overlap = os.path.join(os.path.dirname(src_with_id), os.path.basename(src_with_id).split(f'.{ext}')[0] + "_tiles_overlap")
tilling(src_with_id, src_folder_tiles_w_overlap, tile_size, overlap)

Start tilling (with overlap = 50m)...
Computing the estimated number of tiles...
Done!


100%|██████████| 6/6 [10:38<00:00, 106.48s/it]


In [9]:
# tiles without overlap
src_folder_tiles_wo_overlap = os.path.join(os.path.dirname(src_with_id), os.path.basename(src_with_id).split(f'.{ext}')[0] + "_tiles_no_overlap")
tilling(src_with_id, src_folder_tiles_wo_overlap, tile_size)

Start tilling (with overlap = 0m)...
Computing the estimated number of tiles...
Done!


100%|██████████| 6/6 [14:28<00:00, 144.78s/it]


In [7]:
# Flattening of tiles with overlap
src_folder_flatten_tiles_w_overlap = os.path.join(os.path.dirname(src_with_id), os.path.basename(src_with_id).split('.laz')[0] + "_flatten")
os.makedirs(src_folder_flatten_tiles_w_overlap, exist_ok=True)
flattening(
    src_tiles=src_folder_tiles_w_overlap,
    src_new_tiles=src_folder_flatten_tiles_w_overlap,
    grid_size=grid_size,
    verbose=True,
    do_keep_existing=True,
    verbose_full=False,
)
# flattening(src_tiles, src_new_tiles, grid_size=10, verbose=True, verbose_full=False)

Starting flattening:


Processing: 100%|██████████| 6/6 [20:41<00:00, 206.97s/it]


In [11]:
# Creating flatten tiles w/o overlap
def hash_coords(x, y, z, rounding=2):
    """Create a unique integer hash for each rounded coordinate triple."""
    return np.round(x, rounding) * 1e12 + np.round(y, rounding) * 1e6 + np.round(z, rounding)

list_flatten_to_merge = []
print("Creating flaten tiles without overlap")
list_tiles = [x for x in os.listdir(src_folder_flatten_tiles_w_overlap) if not 'flatten' in x and not 'floor' in x]
for _, tile in tqdm(enumerate(list_tiles), total=len(list_tiles)):
    assert os.path.exists(os.path.join(src_folder_tiles_wo_overlap, tile))

    laz_with_ov = laspy.read(os.path.join(src_folder_flatten_tiles_w_overlap, tile))
    laz_without_ov = laspy.read(os.path.join(src_folder_tiles_wo_overlap, tile))
    laz_flatten_with_ov = laspy.read(os.path.join(src_folder_flatten_tiles_w_overlap, tile.split('.laz')[0] + "_flatten.laz"))

    # with_ov_hash = hash_coords(laz_with_ov.x, laz_with_ov.y, laz_with_ov.z)
    # without_ov_hash = hash_coords(laz_without_ov.x, laz_without_ov.y, laz_without_ov.z)

    mask = np.isin(laz_with_ov.id_point, laz_without_ov.id_point)
    # mask = np.isin(with_ov_hash, without_ov_hash)

    laz_flatten_wo_ov = laz_flatten_with_ov[mask]
    laz_flatten_wo_ov.write(os.path.join(src_folder_flatten_tiles_w_overlap, tile.split('.laz')[0] + "_flatten_no_ov.laz"))
    list_flatten_to_merge.append(os.path.join(src_folder_flatten_tiles_w_overlap, tile.split('.laz')[0] + "_flatten_no_ov.laz"))


Creating flaten tiles without overlap


  0%|          | 0/6 [00:00<?, ?it/s]

Barmasse_2023_AllScans_Raw_Georef_with_id_tile_0_0.laz


 17%|█▋        | 1/6 [00:00<00:02,  2.14it/s]

Barmasse_2023_AllScans_Raw_Georef_with_id_tile_0_1.laz


...

 67%|██████▋   | 4/6 [00:29<00:19,  9.99s/it]

Barmasse_2023_AllScans_Raw_Georef_with_id_tile_2_0.laz


 83%|████████▎ | 5/6 [00:31<00:06,  7.00s/it]

Barmasse_2023_AllScans_Raw_Georef_with_id_tile_2_1.laz


100%|██████████| 6/6 [00:31<00:00,  5.29s/it]


In [12]:
# Merging
print("Merging all flatten tiles together (might take a few minutes)")
src_flatten_file = os.path.join(src_original.split('.laz')[0] + "_flatten.laz")
pipeline_json = {
    "pipeline": list_flatten_to_merge + [
        {"type": "filters.merge"},
        {"type": "writers.las", "filename": src_flatten_file, 'extra_dims': 'all'}
    ]
}

pipeline = pdal.Pipeline(json.dumps(pipeline_json))
pipeline.execute()

Merging all flatten tiles together (might take a few minutes)


113669835

In [9]:
# Generate stripes from the merged flatten
src_folder_stripes = os.path.join(os.path.dirname(src_flatten_file), os.path.basename(src_flatten_file).split('.laz')[0] + "_stripes")
x_span = laz_original.x.max() - laz_original.x.min()
y_span = laz_original.y.max() - laz_original.y.min()
dims = [x_span, y_span]

dims[np.argmin([x_span, y_span])] = stripe_width

print("Creation of stripes:")
stripes_file_fast(src_flatten_file, src_folder_stripes, dims, do_keep_existing=True)

Creation of stripes:
Column 1 / 1:


100%|██████████| 88/88 [00:00<00:00, 21218.67it/s]

88


In [9]:
# Generate stripes from the original
src_folder_stripes = os.path.join(os.path.dirname(src_with_id), os.path.basename(src_with_id).split('.laz')[0] + "_stripes")
x_span = laz_original.x.max() - laz_original.x.min()
y_span = laz_original.y.max() - laz_original.y.min()
dims = [x_span, y_span]

dims[np.argmin([x_span, y_span])] = stripe_width

print("Creation of stripes:")
stripes_file_fast(src_with_id, src_folder_stripes, dims, do_keep_existing=True)

Creation of stripes:
Column 1 / 1:


100%|██████████| 89/89 [00:00<00:00, 9618.97it/s]

89


# Temp

In [16]:
src_barmasse_23 = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\ForestFormer3D\data\ForAINetV2\test_data\small_test_barmasse_2023.laz"
src_barmasse_24 = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\ForestFormer3D\data\ForAINetV2\test_data\other_small_test_downsampled_factor_0.1.laz"
src_lausanne = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\ForestFormer3D\data\ForAINetV2\test_data\SwissSurface_2019_2536_1151_tile_88.laz"

name_place = ["Barmasse 23","Barmasse 24","Lausanne"]
src_places = [src_barmasse_23, src_barmasse_24, src_lausanne]
for i in range(3):
    print(f"{name_place[i]}:")
    laz = laspy.read(src_places[i])
    print(f"\t num points: {len(laz)}")
    print(f"\t X in range: [{min(laz.x)}, {max(laz.x)}]")
    print(f"\t Y in range: [{min(laz.y)}, {max(laz.y)}]")
    print(f"\t Z in range: [{min(laz.z)}, {max(laz.z)}]")
    deltax = int(max(laz.x) - min(laz.x))
    deltay = int(max(laz.y) - min(laz.y))
    deltaz = int(max(laz.z) - min(laz.z))
    print(f"\t Bbox size: [{deltax}, {deltay}, {deltaz}]")


Barmasse 23:
	 num points: 769024
	 X in range: [2587220.0, 2587249.75]
	 Y in range: [1098717.875, 1098748.75]
	 Z in range: [1428.497, 1458.161]
	 Bbox size: [29, 30, 29]
Barmasse 24:
	 num points: 47350
	 X in range: [-502.74, -473.49]
	 Y in range: [-698.26, -671.0500000000001]
	 Z in range: [27.990000000000002, 59.870000000000005]
	 Bbox size: [29, 27, 31]
Lausanne:
	 num points: 162871
	 X in range: [2536804.29, 2536904.2800000003]
	 Y in range: [1151628.73, 1151728.72]
	 Z in range: [377.68, 417.01]
	 Bbox size: [99, 99, 39]


In [19]:
garbage = 36
multi = 27
single = 308

total = garbage + multi + single
print(f"Total: {total}")
print(f"\t Garbage: {garbage} ({int(garbage / total * 100)}%)")
print(f"\t Multi: {multi} ({int(multi / total * 100)}%)")
print(f"\t Single: {single} ({int(single / total * 100)}%)")

Total: 371
	 Garbage: 36 (9%)
	 Multi: 27 (7%)
	 Single: 308 (83%)
